# Profitable App Profiles for the App Store and Google Play Markets
Our aim in this project is to find mobile app profiles that are profitable for the App Store and Google Play Markets. We're working as data analysts for a company that builds Android and iOS mobile apps.

At our company, we only build apps that are free to download and install, and our main source of revenue consists of in-app ads. This means that our revenue for any given app is mostly influenced by the number of users that use our app. Our goal for this project is to analyze data to help our developers understand what kinds of apps are likely to attract more users.

# Opening and Exploring the Data
As of September 2018, there were approximately 2 million iOS apps available on the App Store, and 2.1 million Android apps on Google Play. In this project, we'll analyze a sample of the data instead as collecting data for over ofur million apps reuires a significant amount of time and money. We've found the following the two data sets which would be suitable for our purpose:
- A data set containing data about ten thousand Android apps from Google Play which can be downloaded from this [link](https://dq-content.s3.amazonaws.com/350/googleplaystore.csv).
- A data set containing data about seven thousand iOS apps from the App Store which can be downloaded from this [link](https://dq-content.s3.amazonaws.com/350/AppleStore.csv).

We'll first open the two data sets and continue exploring them further.

In [1]:
from csv import reader
import os

opened_file=open('googleplaystore.csv', encoding="utf8")
read_file = reader(opened_file)
android = list(read_file)
android_header = android[0]
android = android[1:]

opened_file=open('AppleStore.csv', encoding="utf8")
read_file = reader(opened_file)
ios = list(read_file)
ios_header = ios[0]
ios = ios[1:]

We're writing a function named **explore_data()** that we will use to explore the rows in a more simple and readable way. We are also adding an option for our function to show the number of rows and couluns for any data set.

In [2]:
def explore_data(dataset, start, end, rows_and_columns=False):
    dataset_slice=dataset[start:end]
    for row in dataset_slice:
        print(row)
        print('\n')
        
        if rows_and_columns:
            print('Number of rows:', len(dataset))
            print('Number of columns:', len(dataset[0]))

In [3]:
explore_data(android, 1, 4, True)
print(android_header)


['Coloring book moana', 'ART_AND_DESIGN', '3.9', '967', '14M', '500,000+', 'Free', '0', 'Everyone', 'Art & Design;Pretend Play', 'January 15, 2018', '2.0.0', '4.0.3 and up']


Number of rows: 10841
Number of columns: 13
['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


Number of rows: 10841
Number of columns: 13
['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Varies with device', '4.2 and up']


Number of rows: 10841
Number of columns: 13
['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']


We can clearly see that we have 10841 android apps and 13 unique columns in our data set.

In [4]:
explore_data(ios, 1, 4, True)
print(ios_header)

['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1']


Number of rows: 7197
Number of columns: 16
['529479190', 'Clash of Clans', '116476928', 'USD', '0.0', '2130805', '579', '4.5', '4.5', '9.24.12', '9+', 'Games', '38', '5', '18', '1']


Number of rows: 7197
Number of columns: 16
['420009108', 'Temple Run', '65921024', 'USD', '0.0', '1724546', '3842', '4.5', '4.0', '1.6.2', '9+', 'Games', '40', '5', '1', '1']


Number of rows: 7197
Number of columns: 16
['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic']


We can clearly see that we have 7197 iOS apps and 16 unique columns in our data set.

# Deleting Invalid Data
From [one of the discussions](https://www.kaggle.com/lava18/google-play-store-apps/discussion/66015) forums in the Google Play data set, we can see that there's an outlying error for row 10472. We can print that particular row to verify the error and then delete. 

In [5]:
android[10472]
#del android[10472]

['Life Made WI-Fi Touchscreen Photo Frame',
 '1.9',
 '19',
 '3.0M',
 '1,000+',
 'Free',
 '0',
 'Everyone',
 '',
 'February 11, 2018',
 '1.0.19',
 '4.0 and up']

In [6]:
for app in ios:
    if len(app) !=len(ios_header):
        print(app)
        print(ios.index(app))

We can see that there's no such invalid data in the iOS data data set.

# Removing Duplicate Entries

In order to first tackle with the possibiltiy of duplicate apps in our data set, we need to know how many duplicates there are. We do this with the help of the **duplicate_android_apps** list below.

In [7]:
duplicate_android_apps=[]
unique_android_apps=[]

for app in android:
    name=app[0]
    if name in unique_android_apps:
        duplicate_android_apps.append(name)
    else:
        unique_android_apps.append(name)

print("Number of duplicate apps in android:", len(duplicate_android_apps))
print('\n')
print("A few of the duplicate apps in android:", duplicate_android_apps[:10])

Number of duplicate apps in android: 1181


A few of the duplicate apps in android: ['Quick PDF Scanner + OCR FREE', 'Box', 'Google My Business', 'ZOOM Cloud Meetings', 'join.me - Simple Meetings', 'Box', 'Zenefits', 'Google Ads', 'Google My Business', 'Slack']


We see that there's a total of 1181 duplicate apps in our android data set. If these entries are not removed beforehand, they may interefere with our results and give us an incorrect output.

The main difference between the duplicates often occur in the number of ratings they have. This shows us that the data has been collected at different times. Hence, our approach would be to remove all the apps but the one with the highest number of reviews, since it'd mean it's the most recent and the most reliable one out of the bunch.

In order to do that, we will create a dictionary where each key is a unique app name, and the value is the highest number of reviews of that app. This will only have one value per app, that value being the one with the highest number of reviews.

In [8]:
reviews_max = {}

for app in android:
    name = app[0]
    n_reviews = float(app[3])
    
    if name in reviews_max and reviews_max[name] < n_reviews:
        reviews_max[name] = n_reviews
        
    elif name not in reviews_max:
        reviews_max[name] = n_reviews

ValueError: could not convert string to float: '3.0M'

In [ ]:
len(reviews_max)

To remove the duplicates, our objective is to keep the distinct apps with the highest number of reviews.

- We first initialize two lists android_clean and already_added which would store our new cleaned dataset and name of all the distinct apps, respectively.
- Then we loop through the raw android dataset and store the name and number of reviews for each app in name and n_reviews respectively.
- We add the current row (app) to the android_clean list, and the app name (name) to the already_added list if:
    - The number of reviews of the current app matches the number of reviews of that app as described in the reviews_max dictionary; and
    - The name of the app is not already in the already_added list (for when there's multiple entries of app with highest number of ratings)

In [ ]:
android_clean=[]
already_added=[]
for app in android:
    name=app[0]
    n_reviews=float(app[3])
    if (n_reviews == reviews_max[name]) and (name not in already_added):
        android_clean.append(app)
        already_added.append(name)

In [ ]:
print(len(android_clean))
print(len(ios))

# Removing Non-English Apps

We will now remove all those apps which are non-English and we'd do this by using the ASCII table. We'll use this table to see whether it contains any non-ASCII characters or not. We will only remove the app if its name has more than thee non-ASCII characters to let in english apps which have an emoji or certain symbols associated with their name.

In [ ]:
def Englishchar(string):
    c=0
    for char in string:
        if ord(char)>127:
            c+=1
    if c>3:
        return False
        
    return True
print(Englishchar('Docs To Go™ Free Office Suite'))
print(Englishchar('Instachat 😜'))
print(Englishchar('爱奇艺PPS -《欢乐颂2》电视剧热播'))

The function below filters out the non-English apps for both the data sets that we are working with.

In [ ]:
english_android_apps=[]
english_ios_apps=[]

for app in android_clean:
    name=app[0]
    isEnglish=Englishchar(name)
    if isEnglish:
        english_android_apps.append(app)
        
for app in ios:
    name=app[1]
    isEnglish=Englishchar(name)
    if isEnglish:
        english_ios_apps.append(app)

In [ ]:
print(len(english_android_apps))
print(len(english_ios_apps))

# Isolating the Free English-Apps

In our study, since we're focusing only on free English apps, we'll need to isolate the free apps for both our data sets. We do that below.

In [ ]:
free_english_android=[]
free_english_ios=[]

for app in english_android_apps:
    price = app[7]
    if price == '0':
        free_english_android.append(app)
        
for app in english_ios_apps:
    price = app[4]
    if price == '0.0':
        free_english_ios.append(app)
        
print(len(free_english_android))
print(len(free_english_ios))

### Why we want to find an app profile that fits both the App Store and Google Play
Our end goal is to add the app on both Google Play and the App Store. Hence, we need to find app profiles that are successful on both markets.

To minimize risks and overhead, our validation strategy for an app idea is comprised of three steps:
1. Build a minimal Android version of the app, and add it to Google Play.
2. If the app has a good response from users, we develop it further.
3. If the app is profitable after six months, we build an iOS version of the app and add it to the App Store.

We will begin the analysis by getting a sense of the most common genres for each market. For this, we'll build a frequency table for the prime_genre column of the App Store data set, and the Genres and Category columns of the Google Play data set.

In [ ]:
def freq_table(dataset, index):
    freq={}
    c=0
    for app in dataset:
        c+=1
        value=app[index] #value is the name of the app
        if value in freq:
            freq[value]+=1
        else:
            freq[value]=1
     
    freq_percentages={}
    for app in freq:
        value=freq[app]
        freq_percentages[app]=(value/c)*100
    return freq_percentages

def display_table(dataset, index):
    table = freq_table(dataset, index)
    table_display = []
    for key in table:
        key_val_as_tuple = (table[key], key)
        table_display.append(key_val_as_tuple)

    table_sorted = sorted(table_display, reverse = True)
    for entry in table_sorted:
        print(entry[1], ':', entry[0])

In [ ]:
display_table(free_english_ios, 11)

We can see that among the free English apps, more than a half (58.16%) are games. Entertainment apps are close to 8%, followed by photo and video apps, which are close to 5%. Only 3.66% of the apps are designed for education, followed by social networking apps which amount for 3.29% of the apps in our data set.

The general impression is that App Store (at least the part containing free English apps) is dominated by apps that are designed for fun (games, entertainment, photo and video, social networking, sports, music, etc.), while apps with practical purposes (education, shopping, utilities, productivity, lifestyle, etc.) are more rare. However, the fact that fun apps are the most numerous doesn't also imply that they also have the greatest number of users — the demand might not be the same as the offer.

Let's continue by examining the Genres and Category columns of the Google Play data set (two columns which seem to be related).

In [ ]:
display_table(free_english_android, 1)

The landscape seems significantly different on Google Play: there are not that many apps designed for fun, and it seems that a good number of apps are designed for practical purposes (family, tools, business, lifestyle, productivity, etc.). However, if we investigate this further, we can see that the family category (which accounts for almost 19% of the apps) means mostly games for kids.
Even so, practical apps seem to have a better representation on Google Play compared to App Store. This picture is also confirmed by the frequency table we see for the Genres column:



In [ ]:
display_table(free_english_android, -4)

We can notice that the Genres column is much more granular (it has more categories). We're only looking for the bigger picture at the moment, so we'll only work with the Category column moving forward.

Up to this point, we found that the App Store is dominated by apps designed for fun, while Google Play shows a more balanced landscape of both practical and for-fun apps. Now we'd like to get an idea about the kind of apps that have most number of users.

In [ ]:
genre_ios=freq_table(free_english_ios, 11)
for genre in genre_ios:
    total=0
    len_genre=0
    for app in free_english_ios:
        genre_app=app[-5]
        if genre_app==genre:
            ratings=float(app[5])
            total+=ratings
            len_genre+=1
            
    avg_n_of_ratings=total/len_genre
    print(genre, ':', avg_n_of_ratings)

On average, navigation apps have the highest number of user reviews, but this figure is heavily influenced by Waze and Google Maps, which have close to half a million user reviews together. The same pattern can be seen across other genres like social networking or music.





In [ ]:
for app in free_english_ios:
    if app[-5] == 'Navigation':
        print(app[1], ':', app[5]) # print name and number of ratings

Reference apps have 74,942 user ratings on average, but it's actually the Bible and Dictionary.com which skew up the average rating.

However, this niche seems to show some potential. One thing we could do is take another popular book and turn it into an app where we could add different features besides the raw version of the book. This might include daily quotes from the book, an audio version of the book, quizzes about the book, etc. On top of that, we could also embed a dictionary within the app, so users don't need to exit our app to look up words in an external app.

This idea seems to fit well with the fact that the App Store is dominated by for-fun apps. This suggests the market might be a bit saturated with for-fun apps, which means a practical app might have more of a chance to stand out among the huge number of apps on the App Store.

Other genres that seem popular include weather, book, food and drink, or finance. The book genre seem to overlap a bit with the app idea we described above, but the other genres don't seem too interesting to us:

Weather apps — people generally don't spend too much time in-app, and the chances of making profit from in-app adds are low. Also, getting reliable live weather data may require us to connect our apps to non-free APIs.

Food and drink — examples here include Starbucks, Dunkin' Donuts, McDonald's, etc. So making a popular food and drink app requires actual cooking and a delivery service, which is outside the scope of our company.

Finance apps — these apps involve banking, paying bills, money transfer, etc. Building a finance app requires domain knowledge, and we don't want to hire a finance expert just to build an app.

In [ ]:
android_final=freq_table(free_english_android, 1)
for category in android_final:
    total=0
    len_category=0
    for app in free_english_android:
        category_app=app[1]
        if category_app==category:
            installs=app[5]
            installs=installs.replace(',', '')
            installs=installs.replace('+', '')
            total+=float(installs)
            len_category+=1
    avg_installs=total/len_category
    print(category, ':', avg_installs)

On average, communication apps have the most installs: 38,456,119. This number is heavily skewed up by a few apps that have over one billion installs (WhatsApp, Facebook Messenger, Skype, Google Chrome, Gmail, and Hangouts), and a few others with over 100 and 500 million installs. If we removed all the communication apps that have over 100 million installs, the average would be reduced roughly ten times:





In [ ]:
under_100_m = []

for app in free_english_android:
    n_installs = app[5]
    n_installs = n_installs.replace(',', '')
    n_installs = n_installs.replace('+', '')
    if (app[1] == 'COMMUNICATION') and (float(n_installs) < 100000000):
        under_100_m.append(float(n_installs))
        
sum(under_100_m) / len(under_100_m)

We see the same pattern for the video players category, which is the runner-up with 24,727,872 installs. The market is dominated by apps like Youtube, Google Play Movies & TV, or MX Player. The pattern is repeated for social apps (where we have giants like Facebook, Instagram, Google+, etc.), photography apps (Google Photos and other popular photo editors), or productivity apps (Microsoft Word, Dropbox, Google Calendar, Evernote, etc.).

Again, the main concern is that these app genres might seem more popular than they really are. Moreover, these niches seem to be dominated by a few giants who are hard to compete against.

The game genre seems pretty popular, but previously we found out this part of the market seems a bit saturated, so we'd like to come up with a different app recommendation if possible.

The books and reference genre looks fairly popular as well, with an average number of installs of 8,767,811. It's interesting to explore this in more depth, since we found this genre has some potential to work well on the App Store, and our aim is to recommend an app genre that shows potential for being profitable on both the App Store and Google Play.

However, it looks like there are only a few very popular apps, so this market still shows potential. Let's try to get some app ideas based on the kind of apps that are somewhere in the middle in terms of popularity (between 1,000,000 and 100,000,000 downloads):

In [ ]:
for app in free_english_android:
    if app[1] == 'BOOKS_AND_REFERENCE' and (app[5] == '1,000,000+'
                                            or app[5] == '5,000,000+'
                                            or app[5] == '10,000,000+'
                                            or app[5] == '50,000,000+'):
        print(app[0], ':', app[5])

This niche seems to be dominated by software for processing and reading ebooks, as well as various collections of libraries and dictionaries, so it's probably not a good idea to build similar apps since there'll be some significant competition.

We also notice there are quite a few apps built around the book Quran, which suggests that building an app around a popular book can be profitable. It seems that taking a popular book (perhaps a more recent book) and turning it into an app could be profitable for both the Google Play and the App Store markets.

However, it looks like the market is already full of libraries, so we need to add some special features besides the raw version of the book. This might include daily quotes from the book, an audio version of the book, quizzes on the book, a forum where people can discuss the book, etc.

# Conclusions

In this project, we analyzed data about the App Store and Google Play mobile apps with the goal of recommending an app profile that can be profitable for both markets.

We concluded that taking a popular paperback book and turning it into an could be profitable for both the iOS and android markets. Since we would want to stand out from the competition, addition of a few special features besides the raw version of the book would be a good idea. This may include and is not limited to daily quotes from the book, detailed character studies about their story arc, an audio verson of the book and even quizzes on the book as to form a community around it.